# DM - Práctica Text Mining
___

- Universidad Carlos III de Madrid, 4º Curso
- Asignatura: Métodos estadísticos en minería de datos
- Número de grupo: no se que numero somos hahhahahhahahah
- Integrantes: Elena Tran Yang, Marta Madella, Jin Wang y Sabrina Torres Román

In [ ]:
# Instalacion de paquetes
!pip install scikit-learn nltk spacy unidecode networkx wordcloud
!python -m spacy download es_core_news_sm

# Importar librerías
import pandas as pd
from datasets import load_dataset
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer 
import os # útil para manejar ficheros y directorios
import re # útil para manejar expresiones regulares
import nltk # librería de procesamiento de lenguaje natural. Usaremos algunas de sus funciones (stopwords y tokenizers)
import spacy # librería de procesamiento de lenguaje natural. Usaremos algunas de sus funciones (lemmatization)
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from unidecode import unidecode # para eliminar tildes y otros signos diacríticos
import matplotlib.pyplot as plt
import networkx as nx # para crear y manejar grafos
from wordcloud import WordCloud # para crear nubes de palabras

# Descargar recursos adicionales de NLTK
nltk.download('stopwords') # para descargar la lista de stopwords, incluido español
nltk.download('punkt') # para descargar reglas de tokenización
nltk.download('punkt_tab')

# Carga el modelo en español de spaCy que instalamos prevviamente al inicio del notebook
nlp = spacy.load("es_core_news_sm")



In [ ]:
# Cargar dataset
ds = load_dataset("Iker/NoticIA")

df = pd.concat([
ds['train'].to_pandas(),
ds['validation'].to_pandas(),
ds['test'].to_pandas()
], ignore_index=True)